In [1]:
!pip install psutil

You should consider upgrading via the '/Users/melanie/Dev/whylogs-v1/python/.venv/bin/python -m pip install --upgrade pip' command.


In [12]:
import time
import psutil
import whylogs as why

import os

In [15]:
utility_details = {}
tmp_path = os.path.join(os.getcwd(), "example_output")

with why.logger(mode="rolling", interval=2, when="S", base_name="cpu_streaming_data") as logger:
    logger.append_writer("local", base_dir=tmp_path)
    while True:
        utility_details = {"time": time.gmtime(), "cpu_percentage": psutil.cpu_percent()}
        logger.log(utility_details)
        print(utility_details)
        time.sleep(1)

{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=41, tm_sec=49, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 13.1}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=41, tm_sec=50, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 20.2}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=41, tm_sec=51, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 20.7}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=41, tm_sec=52, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 9.5}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=41, tm_sec=53, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 15.2}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=41, tm_sec=54, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 9.4}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_ho

KeyboardInterrupt: 